# Importing .mat data and accessing data in nested arrays

#### About the data

1. data folder contains five directories, one for each animal (T1, S2, Q1, R1, N2)

2. Data is organized in a nested fashion within each animal's folder in the form of `cells` and `MATLAB structures`. A `MATLAB structure` is like a python `dictionary`: it has field names and values. You can access each value using the field name, just like you use keys in a python `dictionary`. A `cell` is like an array in python. It can contain arrays inside it. The first level of data corresponds to each day of recording. The second level corresponds to each epoch of that day. The third level corresponds to each tetrode that was recorded from in that epoch, and the last level corresponds to cells that were clustered from that tetrode. 

3. each directory contains `.mat` files:<br>
    i. `cellinfo.mat` :  has information about each unit that was recorded and clustered: whether it was recorded from the hippocampus (where it will say `CA1`), or from the pre-frontal cortex( where it will say `PFC`).  <br>
    ii. `eeg.mat` : contains raw eeg or local field potential<br>
    iii. `rawpos.mat` : non-interpolated position derived from semiautomated processing of video files. `rawpos{day}{epoch}` has the following fields: <br>
           - data (Nx3) <br>
           - fields (string, gives information about each column of data) <br>
           - cmperpix: size of a video pixel in centimeters. zeros in the data indicate positions where the animal was not tracked and the experimenter did not fill in the location manually. Valid positions are present at least every 3 – 10 frames, depending on the epochs.<br>
    iv. `spikes.mat` : this is the data we're really interested in: this contains the times at with each unit (or clustered neuron) fired an action potential and the x and y coordinates of the animal at that time. spiking data from clustered units. `spikes{day}{epoch}{tetrode}{unit}` has the following fields :
        1. data: (Nx7)
        2. descript: ‘spike data’
        3. fields : identities of columns in data. We suggest that people only use the spike times and re-derive the other fields when needed so they understand how they are generated. 
        4. depth – if specified, this is the number of 1/12ths of a turn of a 0x80 (80 threads / inch) screw from the bottom of the microdrive at which these data were collected. To convert to mm, multiply by 0.0265. 
        5. spikewidth: width, in points sampled at 30 KHz, of spike from peak to trough. To convert to ms, divide by 30. 
        6. timerange: start and end time of epoch in 10KHz units.
    v. `task.mat` : information about the task that the animal was doing during that epoch<br>
    vi. `tetinfo.mat` : information about each tetrode and where it was located in the brain <br>


In [47]:
import scipy.io as spio # you will need this library to import .mat data into python
import numpy as np 
import pandas as pd

In [48]:
matfile = '../data/N2/N2spikes.mat' # lets look at the spike data for rat N2 to begin with

to load the `.mat` file, we will specify and set the `squeeze_me` parameter when using `loadmat` because it makes it easier to access data within each file. Specifically, it uses the `.squeeze()` function to discards any _empty_ dimensions. For instance, if the shape of an array is (10,1) that means it just a one dimensional array. Setting this parameter to true will make it so that any arrays in the data that has shapes like (10,1) become (10,). 

(P.S if you're interested, look at the documentation for [`scipi.io.loadmat`](https://docs.scipy.org/doc/scipy/reference/generated/scipy.io.loadmat.html) for what setting each parameter in the function does)

For instance run the following cells to understand what's happening here:

In [3]:
test_array_1 = np.array([1, 2, 3]) #this is a one-dimensional array
test_array_1.shape

(3,)

In [6]:
test_array_2 = np.array([
    [1, 2, 3, 5], 
    [4, 5, 6, 5], 
    [7, 8, 9, 5]
]) #this is a two-dimensional array.
   # note the number of brackets: [[]]
test_array_2.shape

(3, 4)

In [12]:
test_array_3 = np.array([[
    [1, 2, 3], 
    [4, 5, 6], 
    [7, 8, 9]],
]) 
# this is a three-dimensional array with no data in the third dimension. 
# Think of a cube with a height of 3 (3 rows), a width of 3 (3 columns) and a depth of 0. 
# That's not really a cube, it's just a square with an empty dimension, the depth. 
test_array_3.shape

(1, 3, 3)

In [13]:
# using the .squeeze() function, you can just drop the empty-dimension
# and make your 3 dimensional array with the useless 3rd dimension, into a 2 dimensional array
# essentially, it drops the extra brackets. 

test_array_3_squeezed = test_array_3.squeeze()
test_array_3_squeezed.shape

(3, 3)

**Coming back to our data:**

In [14]:
data = spio.loadmat(matfile, squeeze_me=True) 

# .mat file is loaded as a dictionary

In [15]:
data.keys()

dict_keys(['__header__', '__version__', '__globals__', 'spikes'])

we're only interested in the values associated with the 'spikes' key. 
We know from the data description that the first level of organization of the data is days of recording. So each element of the array `data['spikes']` is going to correspond to one day of recording. 

You can use the `.size` function to determine the number of elements in the array. `.shape` tells you the dimensions. So an array with dimensions (3, 3) has a size of 9. 


In [16]:
test_array_2

array([[1, 2, 3, 5],
       [4, 5, 6, 5],
       [7, 8, 9, 5]])

In [17]:
test_array_2.shape

(3, 4)

In [18]:
test_array_2.size

12

**Coming back to our data**

In [19]:
num_of_days = data['spikes'].size #number of days of recording
num_of_days

10

Next, we know that the second level corresponds to each epoch of recording. We can determine the number of epochs for each day by looking at the size of the array for each day. 

In [20]:
num_of_epochs = data['spikes'][0].size
num_of_epochs

6

The next level of organization is tetrodes recorded from for each epoch. 

In [21]:
num_of_tetrodes = data['spikes'][0][0].size
num_of_tetrodes

0

It looks like there was no usable data recorded during epoch 0 of day 0. Let's try epoch 1. 

In [22]:
num_of_tetrodes = data['spikes'][0][1].size
num_of_tetrodes

19

The next level of organization is clustered units, or neurons from each tetrode. 

In [23]:
# some tetrodes have 0 units, like tetrode 0, and 1
num_of_units = data['spikes'][0][1][1].size
num_of_units

0

In [24]:
# some tetrodes have 1 unit, like tetrode 2
num_of_units = data['spikes'][0][1][2].size
num_of_units

1

In [25]:
# some tetrodes have more than one unit, like tetrode 7
num_of_units = data['spikes'][0][1][7].size
num_of_units

3

Now, let's try to access the data for one unit. Clearly from the indexing we've done so far, to access data from day0, epoch1, tetrode3, unit0, we need to type out the following: 

In [26]:
data['spikes'][0][1][2][0]

IndexError: too many indices for array: array is 0-dimensional, but 1 were indexed

Uh oh, we've encountered a mysterious kind of array that is 0-dimensional. How is this possible? And what does it mean? Let's look up how to access data from it. A quick google search using the error `array is 0-dimensional, but 1 were indexed` led me to [this](https://stackoverflow.com/questions/9814226/error-extracting-element-from-an-array-python) page, where a friendly stranger mentions that:

_a zero-dimensional array, and these cannot be indexed. You also don't need to index it -- you can use a directly as if it were a scalar value....
If you need it in the base type of the array, you can use `a.item()` or `a[()]`._

Let's try that. 

In [27]:
data['spikes'][0][1][2].item()

(array([[ 1315.7409    ,   235.74076814,   157.52148988, ...,
             0.        ,    86.56976121,   202.        ],
        [ 1315.7586    ,   235.74984689,   157.68905883, ...,
             0.        ,    86.16683627,   203.        ],
        [ 1315.7709    ,   235.74984689,   157.68905883, ...,
             0.        ,   101.03786487,   203.        ],
        ...,
        [ 1871.6945    ,   132.99779665,   164.16651562, ...,
             0.        ,    89.12128847, 16858.        ],
        [ 1871.7263    ,   133.49112112,   163.69561382, ...,
             0.        ,    80.02858794, 16859.        ],
        [ 1880.434     ,   143.34175479,   168.40567626, ...,
             0.        ,    92.74707011, 17120.        ]]),
 'spike data',
 'time x y dir not_used amplitude(highest variance channel) posindex',
 59,
 19.68587510118237,
 array([13090000, 19210000]))

Hey! Look at that, we have an array where:
the first element is all the rows and columns of numbers
The second element is the name of the field, which is 'spike data' in this case
the third element is the names of all the columns
the fourth element, as promised in the description is the depth of the tetrode in the brain
the fifth element is the spike width
the sixth element is the times of recording

Let's try to access each of these elements

In [28]:
unit_data = data['spikes'][0][1][2].item() # I'm using a variable unit_data to put this in to make it easier 
                                        # to index so we don't have a giant row of numbers in square brackets

In [34]:
type(unit_data), len(unit_data)

(tuple, 6)

In [29]:
unit_data[0] #should give us the matrix of numbers

array([[ 1315.7409    ,   235.74076814,   157.52148988, ...,
            0.        ,    86.56976121,   202.        ],
       [ 1315.7586    ,   235.74984689,   157.68905883, ...,
            0.        ,    86.16683627,   203.        ],
       [ 1315.7709    ,   235.74984689,   157.68905883, ...,
            0.        ,   101.03786487,   203.        ],
       ...,
       [ 1871.6945    ,   132.99779665,   164.16651562, ...,
            0.        ,    89.12128847, 16858.        ],
       [ 1871.7263    ,   133.49112112,   163.69561382, ...,
            0.        ,    80.02858794, 16859.        ],
       [ 1880.434     ,   143.34175479,   168.40567626, ...,
            0.        ,    92.74707011, 17120.        ]])

In [30]:
unit_data[1] # should give us the data description: spike data

'spike data'

In [31]:
unit_data[2] # should give us the column names

'time x y dir not_used amplitude(highest variance channel) posindex'

In [35]:
unit_data[3]

59

In [36]:
unit_data[4]

19.68587510118237

In [37]:
unit_data[5]

array([13090000, 19210000])

**Let's explore how to access the same data from tetrodes that have more than one unit, like tetrode 7**

In [38]:
tt7 = data['spikes'][0][1][7]

In [44]:
tt7[0].item()[1] # gives us unit 0

'spike data'

In [40]:
tt7[0].item() # gives us unit 0 in that amazing array form that we can easily index

(array([[1.30944940e+03, 2.14700002e+02, 1.53734634e+02, ...,
         0.00000000e+00, 9.15875334e+01, 1.40000000e+01],
        [1.30958680e+03, 2.14313722e+02, 1.54047105e+02, ...,
         0.00000000e+00, 9.50214115e+01, 1.80000000e+01],
        [1.30977950e+03, 2.13836000e+02, 1.54274635e+02, ...,
         0.00000000e+00, 1.03922099e+02, 2.40000000e+01],
        ...,
        [1.87973310e+03, 1.37807415e+02, 1.75410286e+02, ...,
         0.00000000e+00, 1.11613922e+02, 1.70990000e+04],
        [1.87984010e+03, 1.38709917e+02, 1.75672123e+02, ...,
         0.00000000e+00, 7.00386113e+01, 1.71020000e+04],
        [1.88070200e+03, 1.46816326e+02, 1.66923801e+02, ...,
         0.00000000e+00, 9.72451144e+01, 1.71280000e+04]]),
 'spike data',
 'time x y dir not_used amplitude(highest variance channel) posindex',
 73,
 14.592959741602996,
 array([13090000, 19210000]))

In [45]:
tt7[1].item() # gives us unit 1 from tt7

(array([[ 1.31149370e+03,  2.12971499e+02,  1.54410473e+02, ...,
          0.00000000e+00,  2.20009884e+01,  7.50000000e+01],
        [ 1.31390850e+03,  2.30165255e+02,  1.47670736e+02, ...,
          0.00000000e+00,  2.32934047e+01,  1.48000000e+02],
        [ 1.32186240e+03,  2.57213322e+02,  1.79528019e+02, ...,
          0.00000000e+00,  1.95946909e+01,  3.86000000e+02],
        ...,
        [ 1.88096690e+03,  1.51381274e+02,  1.68549626e+02, ...,
          0.00000000e+00, -7.71429920e+00,  1.71360000e+04],
        [ 1.88100030e+03,  1.52041292e+02,  1.68839672e+02, ...,
          0.00000000e+00,  1.31690323e+01,  1.71370000e+04],
        [ 1.88106490e+03,  1.52692267e+02,  1.69072732e+02, ...,
          0.00000000e+00,  4.03500543e+01,  1.71380000e+04]]),
 'spike data',
 'time x y dir not_used amplitude(highest variance channel) posindex',
 73,
 16.069508799669276,
 array([13090000, 19210000]))

In [46]:
tt7[2].item()

(array([[ 1.30914820e+03,  2.15434661e+02,  1.52514585e+02, ...,
          0.00000000e+00,  2.03614747e+01,  5.00000000e+00],
        [ 1.30924790e+03,  2.15105624e+02,  1.52657815e+02, ...,
          0.00000000e+00,  2.80491492e+00,  8.00000000e+00],
        [ 1.30931180e+03,  2.14980072e+02,  1.53055890e+02, ...,
          0.00000000e+00, -8.02339287e+00,  1.00000000e+01],
        ...,
        [ 1.88054350e+03,  1.44563582e+02,  1.67530695e+02, ...,
          0.00000000e+00, -4.92789074e+00,  1.71230000e+04],
        [ 1.88085810e+03,  1.48907088e+02,  1.67381341e+02, ...,
          0.00000000e+00,  5.09602532e+00,  1.71320000e+04],
        [ 1.88100340e+03,  1.52041292e+02,  1.68839672e+02, ...,
          0.00000000e+00,  1.44006662e+01,  1.71370000e+04]]),
 'spike data',
 'time x y dir not_used amplitude(highest variance channel) posindex',
 73,
 16.12842735934616,
 array([13090000, 19210000]))